# BQNT Carry Execution - Setup & Verification

This notebook verifies the BQNT environment and BQL connectivity.

## 1. Import Libraries

In [ ]:
# Standard libraries (pre-installed in BQNT)
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

In [ ]:
# Bloomberg BQL
import bql
bq = bql.Service()
print("BQL Service initialized successfully!")

## 2. Test BQL Query

In [ ]:
# Test basic price fetch
test_ticker = 'EURUSD Curncy'

request = bql.Request(
    test_ticker,
    {'Close': bq.data.px_last()['value']}
)

response = bq.execute(request)
df = response[0].df()

print(f"\nTest Query Result for {test_ticker}:")
print(df)

In [ ]:
# Test historical data fetch
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

request = bql.Request(
    test_ticker,
    {
        'Date': bq.data.px_last()['DATE'],
        'Close': bq.data.px_last()['value'],
    },
    with_params={
        'dates': bq.func.range(
            start_date.strftime('%Y-%m-%d'),
            end_date.strftime('%Y-%m-%d')
        ),
        'fill': 'prev'
    }
)

response = bq.execute(request)
df = pd.concat([d.df() for d in response], axis=1)

print(f"\nHistorical Data (Last 30 Days):")
print(f"Shape: {df.shape}")
print(df.tail())

## 3. Verify All Tickers

In [ ]:
# Test all required tickers
test_tickers = [
    # FX Spot
    'EURUSD Curncy', 'USDJPY Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy',
    # FX Forwards
    'EUR3M Curncy', 'JPY3M Curncy',
    # Metals
    'XAU Curncy', 'XAG Curncy',
    # Treasuries
    'USGG2YR Index', 'USGG10YR Index',
    # Volatility
    'VIX Index',
]

results = []
for ticker in test_tickers:
    try:
        request = bql.Request(ticker, {'Close': bq.data.px_last()['value']})
        response = bq.execute(request)
        df = response[0].df()
        price = df['Close'].iloc[0] if not df.empty else None
        results.append({'Ticker': ticker, 'Price': price, 'Status': '✓'})
    except Exception as e:
        results.append({'Ticker': ticker, 'Price': None, 'Status': f'✗ {str(e)[:30]}'})

results_df = pd.DataFrame(results)
print("\nTicker Verification:")
print(results_df.to_string(index=False))

## 4. Import Carry Execution Modules

In [ ]:
# Add src to path
import sys
sys.path.insert(0, '../src')

# Import modules
from config.tickers import ALL_TICKERS, TRADING_UNIVERSE, get_bloomberg_ticker
from config.parameters import SIGNAL_WEIGHTS, REGIME_PARAMS, get_vix_regime

print("\nConfiguration loaded successfully!")
print(f"\nSignal Weights: {SIGNAL_WEIGHTS}")
print(f"\nTrading Universe: {TRADING_UNIVERSE}")

In [ ]:
# Test data loader
from data.bql_loader import BQLDataLoader
from data.cache import get_cache

cache = get_cache()
loader = BQLDataLoader(cache=cache)

print("BQL Data Loader initialized!")
print(f"Cache size: {cache.size}")

In [ ]:
# Test signal engines
from signals.carry import CarrySignalEngine
from signals.momentum import MomentumSignalEngine
from signals.regime import RegimeSignalEngine

carry_engine = CarrySignalEngine()
momentum_engine = MomentumSignalEngine()
regime_engine = RegimeSignalEngine()

print("Signal engines initialized!")

In [ ]:
# Test portfolio constructor
from portfolio.construction import PortfolioConstructor, PortfolioConfig

portfolio = PortfolioConstructor()
print(f"\nPortfolio Config:")
print(f"  Target Vol: {portfolio.config.target_vol:.0%}")
print(f"  Max Position: {portfolio.config.max_position:.0%}")
print(f"  Risk Parity: {portfolio.config.use_risk_parity}")

## 5. Environment Summary

In [ ]:
print("="*60)
print("BQNT CARRY EXECUTION - ENVIRONMENT VERIFIED")
print("="*60)
print(f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nComponents:")
print("  ✓ BQL Service connected")
print("  ✓ All tickers verified")
print("  ✓ Config modules loaded")
print("  ✓ Data loader ready")
print("  ✓ Signal engines ready")
print("  ✓ Portfolio constructor ready")
print("\nReady to run daily signals!")
print("="*60)